In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts

###  Why Preprocessing is Needed for Laptop Data

Before performing any meaningful analysis or modeling, preprocessing is essential because:

1. **Handling Missing Values**  
   - Laptop datasets often have missing specifications (e.g., `Weight`, `Touchscreen`, `SSD`).  
   - Missing data can skew results or break models.

2. **Correcting Data Types**  
   - Some features like `Ram`, `Weight`, `Price` may be stored as strings.  
   - Converting them to numeric types allows proper calculations and comparisons.

3. **Standardizing Units and Formats**  
   - Memory (`8GB`, `16GB`) or storage (`512GB SSD`) may have mixed formats.  
   - Standardization ensures consistent comparisons.

4. **Encoding Categorical Variables**  
   - Features like `Company`, `TypeName`, `OpSys` are textual.  
   - Encoding them numerically allows models to interpret categorical information.

5. **Removing Redundancy and Noise**  
   - Duplicate rows or irrelevant columns can reduce model accuracy.  
   - Preprocessing ensures cleaner, more meaningful data.

6. **Improving Model Performance**  
   - Clean, consistent, and standardized data leads to better insights and more reliable predictive modeling.


In [ ]:
data = pd.read_excel("../data_source/laptop_data.xlsx")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   TypeName          1303 non-null   object 
 3   Inches            1303 non-null   float64
 4   ScreenResolution  1303 non-null   object 
 5   Cpu               1303 non-null   object 
 6   Ram               1303 non-null   object 
 7   Memory            1303 non-null   object 
 8   Gpu               1303 non-null   object 
 9   OpSys             1303 non-null   object 
 10  Weight            1303 non-null   object 
 11  Price             1303 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 122.3+ KB
Series([], Name: Ram, dtype: object)


In [27]:
data.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')

In [ ]:
data['Ram']

0        8GB
1        8GB
2        8GB
3       16GB
4        8GB
        ... 
1298     4GB
1299    16GB
1300     2GB
1301     6GB
1302     4GB
Name: Ram, Length: 1303, dtype: object

### Why We Clean and Convert the `Ram` Column

1. **Remove Text for Numeric Operations**  
   - Original `Ram` values like `"8GB"` are strings.  
   - You cannot perform calculations, comparisons, or correlations on strings.

2. **Convert to Numeric Type**  
   - After removing `"GB"`, converting to `int32` allows:
     - Sorting by RAM size
     - Computing correlations with `Price` or `Weight`
     - Using in machine learning models

3. **Ensure Consistency**  
   - All values are now pure numbers, avoiding errors during analysis.

In [29]:
data['Ram'] = data['Ram'].str.replace('GB','').astype('int32')
data['Ram']

0        8
1        8
2        8
3       16
4        8
        ..
1298     4
1299    16
1300     2
1301     6
1302     4
Name: Ram, Length: 1303, dtype: int32

In [30]:
data['Weight']

0       1.37kg
1       1.34kg
2       1.86kg
3       1.83kg
4       1.37kg
         ...  
1298     1.8kg
1299     1.3kg
1300     1.5kg
1301    2.19kg
1302     2.2kg
Name: Weight, Length: 1303, dtype: object

### Why We Clean and Convert the `Weight` Column

1. **Remove Units for Numeric Operations**  
   - Original `Weight` values like `"1.37kg"` are strings.  
   - Strings cannot be used for calculations, comparisons, or modeling.

2. **Convert to Float Type**  
   - After removing `"kg"`, converting to `float32` allows:
     - Performing arithmetic operations (e.g., average weight)
     - Comparing weights between laptops
     - Using in machine learning models

3. **Ensure Consistency**  
   - All values are now numeric and standardized, preventing errors in analysis.


In [31]:
data['Weight'] = data['Weight'].str.replace('kg','').astype('float32')
data['Weight']

0       1.37
1       1.34
2       1.86
3       1.83
4       1.37
        ... 
1298    1.80
1299    1.30
1300    1.50
1301    2.19
1302    2.20
Name: Weight, Length: 1303, dtype: float32

In [32]:
data['Price']

0        71378.6832
1        47895.5232
2        30636.0000
3       135195.3360
4        96095.8080
           ...     
1298     33992.6400
1299     79866.7200
1300     12201.1200
1301     40705.9200
1302     19660.3200
Name: Price, Length: 1303, dtype: float64

### Why We Convert the `Price` Column to Integer

1. **Simplify Values**  
   - Original `Price` has decimal points (e.g., `71378.6832`) which are unnecessary for most analyses.  
   - Converting to integer rounds off small fractions, making data cleaner.

2. **Enable Numeric Operations**  
   - Integers are easier to work with for:
     - Sorting and comparisons
     - Aggregations like mean, median, or total
     - Machine learning models that expect numeric input

3. **Consistency Across Dataset**  
   - Aligns `Price` with other integer features like `Ram` for smoother analysis.


In [33]:
data['Price'] = data['Price'].astype(int)
data['Price']

0        71378
1        47895
2        30636
3       135195
4        96095
         ...  
1298     33992
1299     79866
1300     12201
1301     40705
1302     19660
Name: Price, Length: 1303, dtype: int64

In [35]:
data['ScreenResolution']

0               IPS Panel Retina Display 2560x1600
1                                         1440x900
2                                Full HD 1920x1080
3               IPS Panel Retina Display 2880x1800
4               IPS Panel Retina Display 2560x1600
                           ...                    
1298     IPS Panel Full HD / Touchscreen 1920x1080
1299    IPS Panel Quad HD+ / Touchscreen 3200x1800
1300                                      1366x768
1301                                      1366x768
1302                                      1366x768
Name: ScreenResolution, Length: 1303, dtype: object

### Preprocessing `ScreenResolution` and Deriving Features

1. **Extracting Touchscreen and IPS Features**  
   - `Touchscreen` column: 1 if the display mentions "Touchscreen", else 0.  
   - `IPS` column: 1 if the display mentions "IPS", else 0.  
   - Converts textual info into numeric features for modeling.

2. **Splitting Resolution into Width and Height**  
   - Original `ScreenResolution` contains mixed text (e.g., `"IPS Panel Retina Display 2560x1600"`).  
   - Extract `x_res` (width) and `y_res` (height) to get numeric pixel values.

3. **Cleaning and Converting to Numeric**  
   - Remove commas or extra text and convert strings to integers.  
   - Ensures proper numeric calculations.

4. **Calculating PPI (Pixels Per Inch)**  
   - Formula: `sqrt(x_res^2 + y_res^2) / Inches`  
   - Represents screen density, an important feature for laptop quality and pricing.

5. **Dropping Original Columns**  
   - Remove `ScreenResolution`, `Inches`, `x_res`, `y_res` as raw values are no longer needed.  
   - Keeps dataset clean and numeric-only for analysis and modeling.


In [36]:
data['Touchscreen'] = data['ScreenResolution'].apply(lambda x: 1 if 'Touchscreen' in x else 0)
data['Touchscreen']


0       0
1       0
2       0
3       0
4       0
       ..
1298    1
1299    1
1300    0
1301    0
1302    0
Name: Touchscreen, Length: 1303, dtype: int64

In [37]:
data['IPS'] = data['ScreenResolution'].apply(lambda x: 1 if 'IPS' in x else 0)
data['IPS']

0       1
1       0
2       0
3       1
4       1
       ..
1298    1
1299    1
1300    0
1301    0
1302    0
Name: IPS, Length: 1303, dtype: int64

In [39]:
new = data['ScreenResolution'].str.split('x', n=1, expand=True)
new

,0,1
0,IPS Panel Retina Display 2560,1600
1,1440,900
2,Full HD 1920,1080
3,IPS Panel Retina Display 2880,1800
4,IPS Panel Retina Display 2560,1600
...,...,...
1298,IPS Panel Full HD / Touchscreen 1920,1080
1299,IPS Panel Quad HD+ / Touchscreen 3200,1800
1300,1366,768
1301,1366,768


In [40]:
data['x_res'] = new[0]
data['x_res']


0               IPS Panel Retina Display 2560
1                                        1440
2                                Full HD 1920
3               IPS Panel Retina Display 2880
4               IPS Panel Retina Display 2560
                        ...                  
1298     IPS Panel Full HD / Touchscreen 1920
1299    IPS Panel Quad HD+ / Touchscreen 3200
1300                                     1366
1301                                     1366
1302                                     1366
Name: x_res, Length: 1303, dtype: object

In [41]:

data['y_res'] = new[1]
data['y_res'] 

0       1600
1        900
2       1080
3       1800
4       1600
        ... 
1298    1080
1299    1800
1300     768
1301     768
1302     768
Name: y_res, Length: 1303, dtype: object

In [42]:
data['x_res'] = data['x_res'].str.replace(',','').str.findall(r'(\d+\.?\d+)').apply(lambda x: x[0] if x else '0')

In [44]:
data['x_res'] = data['x_res'].astype(int)
data['x_res']

0       2560
1       1440
2       1920
3       2880
4       2560
        ... 
1298    1920
1299    3200
1300    1366
1301    1366
1302    1366
Name: x_res, Length: 1303, dtype: int64

In [45]:
data['y_res'] = data['y_res'].astype(int)
data['y_res']

0       1600
1        900
2       1080
3       1800
4       1600
        ... 
1298    1080
1299    1800
1300     768
1301     768
1302     768
Name: y_res, Length: 1303, dtype: int64

In [46]:
data['ppi'] = (((data['x_res']**2) + (data['y_res']**2))**0.5 / data['Inches']).astype(float)
data['ppi']

0       226.983005
1       127.677940
2       141.211998
3       220.534624
4       226.983005
           ...    
1298    157.350512
1299    276.053530
1300    111.935204
1301    100.454670
1302    100.454670
Name: ppi, Length: 1303, dtype: float64

In [47]:
data.drop(columns=['ScreenResolution','Inches','x_res','y_res'], inplace=True)

In [49]:
data['Cpu'] 

0                       Intel Core i5 2.3GHz
1                       Intel Core i5 1.8GHz
2                 Intel Core i5 7200U 2.5GHz
3                       Intel Core i7 2.7GHz
4                       Intel Core i5 3.1GHz
                        ...                 
1298              Intel Core i7 6500U 2.5GHz
1299              Intel Core i7 6500U 2.5GHz
1300    Intel Celeron Dual Core N3050 1.6GHz
1301              Intel Core i7 6500U 2.5GHz
1302    Intel Celeron Dual Core N3050 1.6GHz
Name: Cpu, Length: 1303, dtype: object

In [50]:
data['Cpu Name'] = data['Cpu'].apply(lambda x: " ".join(x.split()[0:3]))
data['Cpu Name']

0            Intel Core i5
1            Intel Core i5
2            Intel Core i5
3            Intel Core i7
4            Intel Core i5
               ...        
1298         Intel Core i7
1299         Intel Core i7
1300    Intel Celeron Dual
1301         Intel Core i7
1302    Intel Celeron Dual
Name: Cpu Name, Length: 1303, dtype: object

### Standardizing CPU Names into Brands

1. **Purpose of Standardization**  
   - Original `Cpu Name` contains many variations (e.g., `Intel Core i5`, `Intel Core i9`, `Intel Pentium`, `AMD Ryzen 5`).  
   - Too many unique values can make analysis or modeling harder.

2. **Grouping CPUs into Categories**  
   - `"Intel Core i3/i5/i7"` → kept as-is  
   - Other Intel CPUs → `"Other Intel Processor"`  
   - All AMD CPUs → `"AMD Processor"`

3. **Benefits**  
   - Reduces categorical complexity  
   - Makes features more consistent for grouping, encoding, and modeling

4. **Dropping Original Columns**  
   - `Cpu` and `Cpu Name` are removed as their information is now captured in `Cpu brand`.


In [ ]:
def standardize_cpu_names(text):
    if text == 'Intel Core i5' or text == 'Intel Core i7' or text == 'Intel Core i3':
        return text
    else:
        if text.split()[0] == 'Intel':
            return 'Other Intel Processor'
        else:
            return 'AMD Processor'


In [51]:
data['Cpu brand'] = data['Cpu Name'].apply(standardize_cpu_names)
data['Cpu brand'] 

0               Intel Core i5
1               Intel Core i5
2               Intel Core i5
3               Intel Core i7
4               Intel Core i5
                ...          
1298            Intel Core i7
1299            Intel Core i7
1300    Other Intel Processor
1301            Intel Core i7
1302    Other Intel Processor
Name: Cpu brand, Length: 1303, dtype: object

In [ ]:
data.drop(columns=['Cpu','Cpu Name'], inplace=True)\

In [52]:
data['Memory'] = data['Memory'].astype(str).replace('\.0', '', regex=True)
data["Memory"] = data["Memory"].str.replace('GB', '')
data["Memory"] = data["Memory"].str.replace('TB', '000')
new = data["Memory"].str.split("+", n=1, expand=True)
data['Memory'] 

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\kanak\AppData\Local\Temp\ipykernel_28616\3745011071.py:1: SyntaxWarning: invalid escape sequence '\.'
  data['Memory'] = data['Memory'].astype(str).replace('\.0', '', regex=True)


0                 128 SSD
1       128 Flash Storage
2                 256 SSD
3                 512 SSD
4                 256 SSD
              ...        
1298              128 SSD
1299              512 SSD
1300     64 Flash Storage
1301             1000 HDD
1302              500 HDD
Name: Memory, Length: 1303, dtype: object

In [53]:
data["first"] = new[0]
data["first"] = data["first"].str.strip()
data["first"]

0                 128 SSD
1       128 Flash Storage
2                 256 SSD
3                 512 SSD
4                 256 SSD
              ...        
1298              128 SSD
1299              512 SSD
1300     64 Flash Storage
1301             1000 HDD
1302              500 HDD
Name: first, Length: 1303, dtype: object

In [56]:
data["second"] = new[1]

In [58]:
data["Layer1HDD"] = data["first"].apply(lambda x: 1 if "HDD" in x else 0)
data["Layer1SSD"] = data["first"].apply(lambda x: 1 if "SSD" in x else 0)
data["Layer1Hybrid"] = data["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
data["Layer1Flash_Storage"] = data["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)
data["Layer1Flash_Storage"]

0       0
1       1
2       0
3       0
4       0
       ..
1298    0
1299    0
1300    1
1301    0
1302    0
Name: Layer1Flash_Storage, Length: 1303, dtype: int64

In [59]:
# Extract numeric values only
data['first'] = data['first'].str.extract('(\d+)', expand=False)
data['first'].fillna('0', inplace=True)

data["second"].fillna("0", inplace=True)
data["second"]

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\kanak\AppData\Local\Temp\ipykernel_28616\4277038405.py:2: SyntaxWarning: invalid escape sequence '\d'
  data['first'] = data['first'].str.extract('(\d+)', expand=False)
C:\Users\kanak\AppData\Local\Temp\ipykernel_28616\4277038405.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['first'].fillna('0', inplace=True)
C:\Users\kanak\AppData\Local\Temp\ipykernel_28616\4277038405.py:5: FutureWarning: A value is trying t

0       0
1       0
2       0
3       0
4       0
       ..
1298    0
1299    0
1300    0
1301    0
1302    0
Name: second, Length: 1303, dtype: object

In [60]:
data["Layer2HDD"] = data["second"].apply(lambda x: 1 if "HDD" in x else 0)
data["Layer2SSD"] = data["second"].apply(lambda x: 1 if "SSD" in x else 0)
data["Layer2Hybrid"] = data["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
data["Layer2Flash_Storage"] = data["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)

data["Layer2Flash_Storage"]

0       0
1       0
2       0
3       0
4       0
       ..
1298    0
1299    0
1300    0
1301    0
1302    0
Name: Layer2Flash_Storage, Length: 1303, dtype: int64

In [61]:
data['second'] = data['second'].str.extract('(\d+)', expand=False)
data['second'].fillna('0', inplace=True)
data['second']

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\kanak\AppData\Local\Temp\ipykernel_28616\599620497.py:1: SyntaxWarning: invalid escape sequence '\d'
  data['second'] = data['second'].str.extract('(\d+)', expand=False)
C:\Users\kanak\AppData\Local\Temp\ipykernel_28616\599620497.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['second'].fillna('0', inplace=True)


0       0
1       0
2       0
3       0
4       0
       ..
1298    0
1299    0
1300    0
1301    0
1302    0
Name: second, Length: 1303, dtype: object

In [62]:
data["first"] = data["first"].astype(int)
data["first"]

0        128
1        128
2        256
3        512
4        256
        ... 
1298     128
1299     512
1300      64
1301    1000
1302     500
Name: first, Length: 1303, dtype: int64

In [64]:
data["second"] = data["second"].astype(int)
data["second"]

0       0
1       0
2       0
3       0
4       0
       ..
1298    0
1299    0
1300    0
1301    0
1302    0
Name: second, Length: 1303, dtype: int64

In [65]:
data["HDD"] = (data["first"] * data["Layer1HDD"] + data["second"] * data["Layer2HDD"])
data["HDD"] 


0          0
1          0
2          0
3          0
4          0
        ... 
1298       0
1299       0
1300       0
1301    1000
1302     500
Name: HDD, Length: 1303, dtype: int64

In [66]:

data["SSD"] = (data["first"] * data["Layer1SSD"] + data["second"] * data["Layer2SSD"])
data["SSD"]


0       128
1         0
2       256
3       512
4       256
       ... 
1298    128
1299    512
1300      0
1301      0
1302      0
Name: SSD, Length: 1303, dtype: int64

In [67]:

data["Hybrid"] = (data["first"] * data["Layer1Hybrid"] + data["second"] * data["Layer2Hybrid"])
data["Flash_Storage"] = (data["first"] * data["Layer1Flash_Storage"] + data["second"] * data["Layer2Flash_Storage"])


In [68]:
data.drop(columns=['first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
                   'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
                   'Layer2Flash_Storage', 'Memory', 'Hybrid', 'Flash_Storage'], inplace=True)


In [70]:
data['Gpu'] 

0       Intel Iris Plus Graphics 640
1             Intel HD Graphics 6000
2              Intel HD Graphics 620
3                 AMD Radeon Pro 455
4       Intel Iris Plus Graphics 650
                    ...             
1298           Intel HD Graphics 520
1299           Intel HD Graphics 520
1300               Intel HD Graphics
1301              AMD Radeon R5 M330
1302               Intel HD Graphics
Name: Gpu, Length: 1303, dtype: object

### Preprocessing `Gpu` Column

1. **Extract GPU Brand**  
   - Original `Gpu` values contain full names (e.g., `"Intel Iris Plus Graphics 640"`, `"AMD Radeon Pro 455"`).  
   - `Gpu brand` keeps only the first word (brand), e.g., `"Intel"` or `"AMD"`.

2. **Remove Irrelevant Entries**  
   - Rows with `"ARM"` GPUs are removed as they are uncommon or not relevant for analysis.

3. **Drop Original Column**  
   - `Gpu` is dropped after extracting the brand, keeping dataset cleaner and categorical features consistent.


In [ ]:
data['Gpu brand'] = data['Gpu'].apply(lambda x: x.split()[0])
data = data[data['Gpu brand'] != 'ARM']
data.drop(columns=['Gpu'], inplace=True)


In [72]:
data['OpSys']

0            macOS
1            macOS
2            No OS
3            macOS
4            macOS
           ...    
1298    Windows 10
1299    Windows 10
1300    Windows 10
1301    Windows 10
1302    Windows 10
Name: OpSys, Length: 1303, dtype: object

### Preprocessing `OpSys` Column

1. **Original `OpSys` is Too Detailed**  
   - Values include `"Windows 10"`, `"Windows 7"`, `"macOS"`, `"No OS"`, `"Linux"`, etc.  
   - Too many unique categories can complicate analysis and modeling.

2. **Mapping to Simplified Categories**  
   - Windows variants → `"windows"`  
   - macOS variants → `"Mac"`  
   - Others (Linux, No OS) → `"Other/Linux/No OS"`  

3. **Benefits**  
   - Reduces complexity of categorical data  
   - Makes feature more consistent for grouping, encoding, and modeling.

4. **Drop Original Column**  
   - `OpSys` is removed after mapping since the simplified `os` captures the essential information.


In [21]:
def map_operating_system(inp):
    if inp == 'Windows 10' or inp == 'Windows 7' or inp == 'Windows 10 S':
        return 'windows'
    elif inp == 'macOS' or inp == 'Mac OS X':
        return 'Mac'
    else:
        return 'Other/Linux/No OS'


In [22]:
data['os'] = data['OpSys'].apply(map_operating_system)
data.drop(columns=['OpSys'], inplace=True)


In [23]:
data.to_csv("../data_source/laptop_data_preprocessed.csv", index=False)
